# RAG 평가(Evaluation) 튜토리얼

## 왜 평가가 RAG 아키텍처보다 더 중요한가?

RAG(Retrieval-Augmented Generation) 시스템을 구축할 때 많은 개발자들이 **아키텍처 선택**에 집중합니다:
- 어떤 임베딩 모델을 사용할까?
- 어떤 벡터 데이터베이스가 좋을까?
- 청킹 전략은 어떻게 할까?

하지만 진짜 중요한 질문은 따로 있습니다:

> **"우리 RAG 시스템이 실제로 잘 작동하는지 어떻게 알 수 있는가?"**

### 평가가 중요한 이유

| 관점 | 평가 없이 | 평가 있으면 |
|------|----------|------------|
| **개발** | 감으로 튜닝 | 데이터 기반 개선 |
| **배포** | 불안한 출시 | 자신있는 릴리즈 |
| **비즈니스** | ROI 측정 불가 | 명확한 성과 지표 |
| **유지보수** | 회귀 버그 발견 어려움 | 지속적인 품질 모니터링 |

### 비즈니스 관점에서의 평가 필요성

1. **비용 정당화**: RAG 시스템의 성능을 수치로 증명
2. **리스크 관리**: 잘못된 정보 제공 시 발생하는 비용 최소화
3. **지속적 개선**: A/B 테스트를 통한 점진적 향상
4. **고객 신뢰**: 일관된 품질 보장

---

## 학습 목표

이 노트북에서는 다음을 배웁니다:

1. **테스트 데이터 구조** 설계 방법
2. **검색 평가 (Retrieval Evaluation)**: MRR, nDCG, Keyword Coverage
3. **답변 평가 (Answer Evaluation)**: LLM-as-a-Judge 패러다임
4. **실전 평가** 파이프라인 구현

---

## 1. 환경 설정

In [ ]:
# 필요한 라이브러리 설치
# !pip install pydantic litellm python-dotenv numpy

In [ ]:
import os
import json
import numpy as np
from typing import List, Dict, Optional, Literal
from pydantic import BaseModel, Field
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv(override=True)

# API 키 확인
if os.getenv("OPENAI_API_KEY"):
    print("OpenAI API 키가 설정되었습니다.")
else:
    print("경고: OPENAI_API_KEY가 설정되지 않았습니다. .env 파일을 확인하세요.")

---

## 2. 테스트 데이터 구조

### 왜 테스트 데이터가 필요한가?

RAG 시스템을 평가하려면 **정답이 있는 테스트 셋**이 필요합니다:

```
테스트 데이터 = 질문 + 예상 답변 + 관련 문서 정보
```

좋은 테스트 데이터의 특징:
- **다양한 유형**의 질문 포함
- **현실적인** 시나리오 반영
- **명확한 정답** 존재
- **충분한 양** (최소 50-100개 권장)

### TestQuestion 클래스 (Pydantic 모델)

In [ ]:
# 질문 카테고리 타입 정의
QuestionCategory = Literal[
    "direct_fact",    # 직접적인 사실 질문
    "temporal",       # 시간 관련 질문
    "spanning",       # 여러 문서에 걸친 질문
    "comparative",    # 비교 질문
    "numerical",      # 숫자/수치 질문
    "relationship",   # 관계 질문
    "holistic"        # 종합적 이해 질문
]


class TestQuestion(BaseModel):
    """
    RAG 평가를 위한 테스트 질문 모델
    
    각 필드 설명:
    - id: 질문 고유 식별자
    - question: 실제 질문 텍스트
    - expected_answer: 예상되는 정답 (참조 답변)
    - category: 질문 카테고리
    - source_docs: 정답을 찾을 수 있는 문서 ID들
    - keywords: 답변에 포함되어야 할 핵심 키워드들
    - difficulty: 난이도 (1: 쉬움, 2: 보통, 3: 어려움)
    """
    id: str = Field(..., description="질문 고유 식별자")
    question: str = Field(..., description="질문 텍스트")
    expected_answer: str = Field(..., description="예상 정답")
    category: QuestionCategory = Field(..., description="질문 카테고리")
    source_docs: List[str] = Field(default_factory=list, description="정답 소스 문서 ID들")
    keywords: List[str] = Field(default_factory=list, description="핵심 키워드들")
    difficulty: int = Field(default=1, ge=1, le=3, description="난이도 (1-3)")


# 예시 테스트 질문 생성
sample_question = TestQuestion(
    id="q001",
    question="Python에서 리스트를 정렬하는 방법은 무엇인가요?",
    expected_answer="Python에서 리스트를 정렬하려면 sort() 메서드 또는 sorted() 함수를 사용합니다. sort()는 원본 리스트를 직접 수정하고, sorted()는 새로운 정렬된 리스트를 반환합니다.",
    category="direct_fact",
    source_docs=["python_basics_001", "python_list_002"],
    keywords=["sort", "sorted", "리스트", "정렬"],
    difficulty=1
)

print("테스트 질문 예시:")
print(sample_question.model_dump_json(indent=2))

### 질문 카테고리 설명

다양한 카테고리의 질문으로 테스트해야 RAG 시스템의 전체적인 성능을 파악할 수 있습니다.

| 카테고리 | 설명 | 예시 | 권장 비율 |
|----------|------|------|----------|
| **direct_fact** | 직접적인 사실 질문 | "Python의 창시자는?" | 45-50% |
| **temporal** | 시간 관련 질문 | "Python 3.0은 언제 출시됐나?" | 10-15% |
| **spanning** | 여러 문서 연결 | "Django와 Flask의 차이점은?" | 10-15% |
| **comparative** | 비교 질문 | "어떤 방법이 더 빠른가?" | 5-10% |
| **numerical** | 수치 관련 | "최대 연결 수는?" | 5-10% |
| **relationship** | 관계 질문 | "A는 B와 어떤 관계?" | 5-10% |
| **holistic** | 종합 이해 | "전체 아키텍처를 설명해주세요" | 5-10% |

In [ ]:
# 샘플 테스트 데이터셋 생성
sample_test_data = [
    TestQuestion(
        id="q001",
        question="Python에서 리스트를 정렬하는 방법은?",
        expected_answer="sort() 메서드 또는 sorted() 함수를 사용합니다.",
        category="direct_fact",
        source_docs=["doc_001"],
        keywords=["sort", "sorted"],
        difficulty=1
    ),
    TestQuestion(
        id="q002",
        question="Python 3.0은 언제 출시되었나요?",
        expected_answer="Python 3.0은 2008년 12월 3일에 출시되었습니다.",
        category="temporal",
        source_docs=["doc_002"],
        keywords=["2008", "12월"],
        difficulty=1
    ),
    TestQuestion(
        id="q003",
        question="Django와 Flask의 주요 차이점은 무엇인가요?",
        expected_answer="Django는 풀스택 프레임워크로 ORM, 인증 등을 내장하고, Flask는 마이크로 프레임워크로 최소한의 기능만 제공하여 유연성이 높습니다.",
        category="spanning",
        source_docs=["doc_003", "doc_004"],
        keywords=["풀스택", "마이크로", "ORM", "유연성"],
        difficulty=2
    ),
    TestQuestion(
        id="q004",
        question="리스트 컴프리헨션과 for 루프 중 어떤 것이 더 빠른가요?",
        expected_answer="일반적으로 리스트 컴프리헨션이 for 루프보다 빠릅니다. 이는 내부적으로 최적화되어 있기 때문입니다.",
        category="comparative",
        source_docs=["doc_005"],
        keywords=["리스트 컴프리헨션", "빠르", "최적화"],
        difficulty=2
    ),
    TestQuestion(
        id="q005",
        question="Python의 기본 재귀 깊이 제한은 얼마인가요?",
        expected_answer="Python의 기본 재귀 깊이 제한은 1000입니다.",
        category="numerical",
        source_docs=["doc_006"],
        keywords=["1000", "재귀"],
        difficulty=1
    ),
]

print(f"테스트 데이터셋: {len(sample_test_data)}개 질문")
for q in sample_test_data:
    print(f"  [{q.category}] {q.question[:40]}...")

---

## 3. 검색 평가 (Retrieval Evaluation)

검색 평가는 RAG의 첫 번째 단계인 **문서 검색**의 품질을 측정합니다.

핵심 질문:
> "검색된 문서가 질문에 대한 답을 찾기에 충분한가?"

### 3.1 MRR (Mean Reciprocal Rank)

**핵심 아이디어**: "첫 번째 정답이 몇 번째에 나왔는가?"

#### 수식

$$MRR = \frac{1}{|Q|} \times \sum_{i=1}^{|Q|} \frac{1}{rank_i}$$

- $|Q|$: 전체 쿼리 수
- $rank_i$: i번째 쿼리에서 첫 번째 정답의 순위

#### 점수 해석

| 정답 위치 | 점수 |
|-----------|------|
| 1등 | 1/1 = **1.0** |
| 2등 | 1/2 = **0.5** |
| 3등 | 1/3 = **0.33** |
| 10등 | 1/10 = **0.1** |
| 정답 없음 | **0.0** |

In [ ]:
def calculate_mrr(retrieved_docs: List[str], relevant_docs: List[str]) -> float:
    """
    MRR (Mean Reciprocal Rank) 계산
    
    단일 쿼리에 대한 Reciprocal Rank를 계산합니다.
    여러 쿼리의 평균을 구하면 MRR이 됩니다.
    
    Args:
        retrieved_docs: 검색된 문서 ID 리스트 (순서대로)
        relevant_docs: 관련 문서(정답) ID 리스트
    
    Returns:
        첫번째 정답에 대해서만 고려함.
        Reciprocal Rank (0~1, 정답이 없으면 0)
    """
    relevant_set = set(relevant_docs)
    
    for rank, doc_id in enumerate(retrieved_docs, start=1):
        if doc_id in relevant_set:
            return 1.0 / rank
    
    return 0.0  # 정답 문서를 찾지 못한 경우


# 예시
retrieved = ["doc_A", "doc_B", "doc_C", "doc_D", "doc_E"]
relevant = ["doc_C", "doc_F"]  # 정답 문서들

mrr = calculate_mrr(retrieved, relevant)
print(f"검색된 문서: {retrieved}")
print(f"정답 문서: {relevant}")
print(f"Reciprocal Rank: {mrr:.3f}")
print(f"\n해석: 첫 번째 정답(doc_C)이 3번째에 있으므로 RR = 1/3 = 0.333")

### 3.2 nDCG (Normalized Discounted Cumulative Gain)

**핵심 아이디어**: "관련성 높은 문서가 위에 있을수록 좋다"

MRR과의 차이점:
- MRR: 첫 번째 정답만 고려
- nDCG: **여러 정답의 순서**까지 고려

#### 계산 과정

1. **DCG (Discounted Cumulative Gain)** 계산:
$$DCG = \sum_{i=1}^{k} \frac{rel_i}{\log_2(i+1)}$$

2. **IDCG (Ideal DCG)** 계산:
   - 관련성 점수를 내림차순 정렬 후 DCG 계산

3. **nDCG** 계산:
$$nDCG = \frac{DCG}{IDCG}$$

#### Binary Relevance

RAG 평가에서는 주로 **binary relevance** (0 또는 1)를 사용:
- 관련 문서: 1
- 비관련 문서: 0

In [ ]:
def calculate_dcg(relevances: List[int], k: int = None) -> float:
    """
    DCG (Discounted Cumulative Gain) 계산
    
    Args:
        relevances: 각 문서의 관련성 점수 (순서대로)
        k: 상위 k개만 고려 (None이면 전체)
    
    Returns:
        DCG 점수
    """
    if k is None:
        k = len(relevances)
    
    dcg = 0.0
    for i in range(min(k, len(relevances))):
        # 위치 i+1의 할인율 적용 (로그 베이스 2)
        dcg += relevances[i] / np.log2(i + 2)  # i+2는 position이 1부터 시작하기 때문
    
    return dcg


def calculate_ndcg(retrieved_docs: List[str], relevant_docs: List[str], k: int = None) -> float:
    """
    nDCG (Normalized DCG) 계산 - Binary Relevance 버전
    
    Args:
        retrieved_docs: 검색된 문서 ID 리스트 (순서대로)
        relevant_docs: 관련 문서(정답) ID 리스트
        k: 상위 k개만 고려
    
    Returns:
        nDCG 점수 (0~1)
    """
    if k is None:
        k = len(retrieved_docs)
    
    relevant_set = set(relevant_docs)
    
    # Binary relevance 계산
    relevances = [1 if doc in relevant_set else 0 for doc in retrieved_docs[:k]]
    
    # 실제 DCG 계산
    dcg = calculate_dcg(relevances, k)
    
    # 이상적인 순서: 관련 문서가 모두 앞에 오는 경우
    ideal_relevances = sorted(relevances, reverse=True)
    idcg = calculate_dcg(ideal_relevances, k)
    
    # IDCG가 0이면 관련 문서가 없음
    if idcg == 0:
        return 0.0
    
    return dcg / idcg


# 예시
retrieved = ["doc_A", "doc_B", "doc_C", "doc_D", "doc_E"]
relevant = ["doc_A", "doc_C", "doc_E"]  # 1, 3, 5번째가 정답

ndcg = calculate_ndcg(retrieved, relevant, k=5)
print(f"검색된 문서: {retrieved}")
print(f"정답 문서: {relevant}")
print(f"\nRelevance 패턴: [1, 0, 1, 0, 1]")
print(f"이상적 패턴:    [1, 1, 1, 0, 0]")
print(f"\nnDCG@5: {ndcg:.3f}")
print(f"\n해석: 정답 문서들이 완벽한 순서가 아니므로 1.0보다 낮음")

### 3.3 Keyword Coverage

**핵심 아이디어**: "검색된 문서에서 핵심 키워드가 얼마나 발견되는가?"

LLM 기반 평가 없이 빠르게 검색 품질을 확인할 수 있는 휴리스틱 지표입니다.

$$Keyword\ Coverage = \frac{\text{발견된 키워드 수}}{\text{전체 키워드 수}}$$

In [ ]:
def calculate_keyword_coverage(retrieved_content: str, keywords: List[str]) -> float:
    """
    키워드 커버리지 계산
    
    Args:
        retrieved_content: 검색된 문서들의 내용 (연결된 텍스트)
        keywords: 찾아야 할 키워드 리스트
    
    Returns:
        커버리지 비율 (0~1)
    """
    if not keywords:
        return 1.0  # 키워드가 없으면 100%로 간주
    
    content_lower = retrieved_content.lower()
    found_count = sum(1 for kw in keywords if kw.lower() in content_lower)
    
    return found_count / len(keywords)


# 예시
document_content = """
Python에서 리스트를 정렬하는 방법은 두 가지가 있습니다.
sort() 메서드는 원본 리스트를 직접 수정합니다.
sorted() 함수는 새로운 정렬된 리스트를 반환합니다.
"""

keywords = ["sort", "sorted", "리스트", "정렬", "reverse"]  # reverse는 없음

coverage = calculate_keyword_coverage(document_content, keywords)
print(f"키워드: {keywords}")
print(f"Keyword Coverage: {coverage:.1%}")
print(f"\n해석: 5개 중 4개 키워드가 발견됨 (reverse 누락)")

### 3.4 RetrievalEval 종합 클래스

모든 검색 평가 지표를 하나로 묶은 Pydantic 모델입니다.

In [ ]:
class RetrievalEval(BaseModel):
    """
    검색 평가 결과 모델
    
    RAG 시스템의 검색 단계 성능을 종합적으로 평가합니다.
    """
    mrr: float = Field(..., ge=0, le=1, description="Mean Reciprocal Rank")
    ndcg: float = Field(..., ge=0, le=1, description="Normalized DCG")
    keyword_coverage: float = Field(..., ge=0, le=1, description="키워드 커버리지")
    retrieved_count: int = Field(..., ge=0, description="검색된 문서 수")
    relevant_found: int = Field(..., ge=0, description="찾은 관련 문서 수")
    
    @property
    def overall_score(self) -> float:
        """종합 점수 (가중 평균)"""
        return 0.4 * self.mrr + 0.4 * self.ndcg + 0.2 * self.keyword_coverage


def evaluate_retrieval(
    retrieved_docs: List[str],
    relevant_docs: List[str],
    retrieved_content: str,
    keywords: List[str],
    k: int = 5
) -> RetrievalEval:
    """
    검색 평가 수행
    
    Args:
        retrieved_docs: 검색된 문서 ID 리스트
        relevant_docs: 관련 문서(정답) ID 리스트
        retrieved_content: 검색된 문서들의 내용
        keywords: 핵심 키워드 리스트
        k: nDCG 계산에 사용할 k
    
    Returns:
        RetrievalEval 객체
    """
    relevant_set = set(relevant_docs)
    relevant_found = sum(1 for doc in retrieved_docs if doc in relevant_set)
    
    return RetrievalEval(
        mrr=calculate_mrr(retrieved_docs, relevant_docs),
        ndcg=calculate_ndcg(retrieved_docs, relevant_docs, k),
        keyword_coverage=calculate_keyword_coverage(retrieved_content, keywords),
        retrieved_count=len(retrieved_docs),
        relevant_found=relevant_found
    )


# 종합 평가 예시
eval_result = evaluate_retrieval(
    retrieved_docs=["doc_A", "doc_B", "doc_C", "doc_D", "doc_E"],
    relevant_docs=["doc_A", "doc_C"],
    retrieved_content="Python에서 sort()와 sorted() 함수를 사용하여 리스트를 정렬할 수 있습니다.",
    keywords=["sort", "sorted", "리스트", "정렬"],
    k=5
)

print("검색 평가 결과:")
print(f"  MRR: {eval_result.mrr:.3f}")
print(f"  nDCG@5: {eval_result.ndcg:.3f}")
print(f"  Keyword Coverage: {eval_result.keyword_coverage:.1%}")
print(f"  관련 문서 발견: {eval_result.relevant_found}/{len(eval_result.retrieved_count)}개 중")
print(f"\n종합 점수: {eval_result.overall_score:.3f}")

---

## 4. 답변 평가 (Answer Evaluation)

검색이 잘 되었더라도, **최종 답변의 품질**이 좋아야 합니다.

### 4.1 LLM-as-a-Judge 패러다임

**핵심 아이디어**: LLM을 평가자로 사용하여 답변 품질 측정

#### 장점
- **스케일러블**: 대량의 평가를 자동화
- **일관성**: 동일 기준으로 평가
- **세밀한 피드백**: 왜 좋은지/나쁜지 설명 가능
- **다양한 기준**: 정확도, 완전성, 관련성 등 여러 측면 평가

#### 단점
- **비용**: API 호출 비용 발생
- **편향 가능성**: LLM 자체의 편향 반영
- **일관성 한계**: 같은 입력에 다른 결과 가능

### 4.2 평가 기준 3가지

| 기준 | 설명 | 예시 질문 |
|------|------|----------|
| **Accuracy (정확도)** | 사실적으로 얼마나 정확한가? | "답변이 사실과 일치하나요?" |
| **Completeness (완전성)** | 질문의 모든 측면을 다루는가? | "모든 부분을 답변했나요?" |
| **Relevance (관련성)** | 질문에 직접적으로 답변하는가? | "질문에 맞는 답변인가요?" |

In [ ]:
class AnswerEval(BaseModel):
    """
    답변 평가 결과 모델
    
    LLM-as-a-Judge가 생성한 평가 결과를 구조화합니다.
    """
    accuracy: int = Field(..., ge=1, le=5, description="정확도 점수 (1-5)")
    completeness: int = Field(..., ge=1, le=5, description="완전성 점수 (1-5)")
    relevance: int = Field(..., ge=1, le=5, description="관련성 점수 (1-5)")
    feedback: str = Field(..., description="상세 피드백")
    
    @property
    def average_score(self) -> float:
        """평균 점수"""
        return (self.accuracy + self.completeness + self.relevance) / 3
    
    @property
    def normalized_score(self) -> float:
        """정규화된 점수 (0-1 범위)"""
        return (self.average_score - 1) / 4  # 1-5를 0-1로 변환


# 예시 평가 결과
sample_eval = AnswerEval(
    accuracy=4,
    completeness=3,
    relevance=5,
    feedback="답변이 질문에 잘 맞지만, 추가 예제가 있으면 더 좋겠습니다."
)

print("답변 평가 예시:")
print(f"  정확도: {sample_eval.accuracy}/5")
print(f"  완전성: {sample_eval.completeness}/5")
print(f"  관련성: {sample_eval.relevance}/5")
print(f"  평균 점수: {sample_eval.average_score:.2f}/5")
print(f"  정규화 점수: {sample_eval.normalized_score:.2f}")
print(f"  피드백: {sample_eval.feedback}")

### 4.3 Judge 프롬프트 설계

LLM-as-a-Judge의 핵심은 **잘 설계된 프롬프트**입니다.

In [ ]:
JUDGE_SYSTEM_PROMPT = """
당신은 RAG 시스템의 답변 품질을 평가하는 전문 평가자입니다.

주어진 질문, 생성된 답변, 참조 답변을 비교하여 다음 세 가지 기준으로 평가하세요:

1. **정확도 (Accuracy)** [1-5점]
   - 5점: 완전히 정확함, 사실적 오류 없음
   - 4점: 대부분 정확함, 사소한 오류
   - 3점: 부분적으로 정확함, 일부 오류
   - 2점: 많은 오류 포함
   - 1점: 대부분 부정확함

2. **완전성 (Completeness)** [1-5점]
   - 5점: 질문의 모든 측면을 완벽히 다룸
   - 4점: 대부분의 측면을 다룸
   - 3점: 핵심만 다룸, 일부 누락
   - 2점: 많은 부분 누락
   - 1점: 거의 답변하지 않음

3. **관련성 (Relevance)** [1-5점]
   - 5점: 질문에 직접적이고 정확하게 답변
   - 4점: 대부분 관련있는 답변
   - 3점: 부분적으로 관련있음
   - 2점: 관련성 낮음
   - 1점: 완전히 동떨어진 답변

반드시 JSON 형식으로 응답하세요.
"""


def create_judge_user_prompt(question: str, generated_answer: str, reference_answer: str) -> str:
    """Judge에게 전달할 User 프롬프트 생성"""
    return f"""
## 평가 대상

### 질문
{question}

### 생성된 답변 (평가 대상)
{generated_answer}

### 참조 답변 (정답)
{reference_answer}

## 요청

위 정보를 바탕으로 생성된 답변을 평가해주세요.
"""


# 프롬프트 예시 출력
print("=" * 60)
print("System Prompt:")
print("=" * 60)
print(JUDGE_SYSTEM_PROMPT[:500] + "...")
print("\n" + "=" * 60)
print("User Prompt 예시:")
print("=" * 60)
print(create_judge_user_prompt(
    question="Python에서 리스트 정렬 방법은?",
    generated_answer="sort() 함수를 사용합니다.",
    reference_answer="sort() 메서드와 sorted() 함수를 사용할 수 있습니다."
))

### 4.4 LLM Judge 구현

LiteLLM을 사용하여 다양한 LLM 제공자를 지원하는 Judge를 구현합니다.

In [ ]:
from litellm import completion
import json


def evaluate_answer_with_llm(
    question: str,
    generated_answer: str,
    reference_answer: str,
    model: str = "gpt-4o-mini"
) -> AnswerEval:
    """
    LLM을 사용하여 답변 품질 평가
    
    Args:
        question: 원본 질문
        generated_answer: RAG 시스템이 생성한 답변
        reference_answer: 정답 (기대 답변)
        model: 사용할 LLM 모델
    
    Returns:
        AnswerEval 객체
    """
    user_prompt = create_judge_user_prompt(question, generated_answer, reference_answer)
    
    # Structured Output을 위한 response_format
    response = completion(
        model=model,
        messages=[
            {"role": "system", "content": JUDGE_SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt}
        ],
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "answer_evaluation",
                "strict": True,
                "schema": {
                    "type": "object",
                    "properties": {
                        "accuracy": {
                            "type": "integer",
                            "description": "정확도 점수 (1-5)"
                        },
                        "completeness": {
                            "type": "integer",
                            "description": "완전성 점수 (1-5)"
                        },
                        "relevance": {
                            "type": "integer",
                            "description": "관련성 점수 (1-5)"
                        },
                        "feedback": {
                            "type": "string",
                            "description": "상세 피드백"
                        }
                    },
                    "required": ["accuracy", "completeness", "relevance", "feedback"],
                    "additionalProperties": False
                }
            }
        },
        temperature=0  # 일관된 평가를 위해 temperature 0
    )
    
    # JSON 파싱
    result = json.loads(response.choices[0].message.content)
    
    return AnswerEval(**result)


print("LLM Judge 함수가 정의되었습니다.")
print("\n다음 섹션에서 실제 평가를 실행합니다.")

---

## 5. 실습: 평가 실행

지금까지 배운 내용을 종합하여 실제 평가를 수행해봅니다.

In [ ]:
# 테스트 시나리오 설정
test_case = {
    "question": "Python에서 리스트를 정렬하는 방법은 무엇인가요?",
    "reference_answer": "Python에서 리스트를 정렬하려면 sort() 메서드 또는 sorted() 함수를 사용합니다. sort()는 원본 리스트를 직접 수정하고, sorted()는 새로운 정렬된 리스트를 반환합니다.",
    "keywords": ["sort", "sorted", "리스트", "정렬"],
    "relevant_docs": ["python_list_001", "python_basics_002"]
}

# 가상의 RAG 시스템 출력 (실제로는 RAG 파이프라인에서 생성)
rag_output = {
    "retrieved_docs": ["python_list_001", "python_intro_003", "python_basics_002", "django_001", "flask_002"],
    "retrieved_content": """Python에서 리스트 정렬은 sort() 메서드를 사용합니다. 
    이 메서드는 원본 리스트를 직접 수정합니다. sorted() 함수도 있습니다.""",
    "generated_answer": "Python에서 리스트를 정렬하려면 sort() 메서드를 사용합니다. 이 메서드는 원본 리스트를 직접 수정합니다."
}

print("테스트 케이스:")
print(f"  질문: {test_case['question']}")
print(f"  정답 문서: {test_case['relevant_docs']}")
print(f"\nRAG 출력:")
print(f"  검색된 문서: {rag_output['retrieved_docs']}")
print(f"  생성된 답변: {rag_output['generated_answer'][:50]}...")

In [ ]:
# 검색 평가 실행
retrieval_eval = evaluate_retrieval(
    retrieved_docs=rag_output["retrieved_docs"],
    relevant_docs=test_case["relevant_docs"],
    retrieved_content=rag_output["retrieved_content"],
    keywords=test_case["keywords"],
    k=5
)

print("=" * 60)
print("검색 평가 결과 (Retrieval Evaluation)")
print("=" * 60)
print(f"\nMRR: {retrieval_eval.mrr:.3f}")
print(f"  -> 첫 번째 정답(python_list_001)이 1번째에 있어 최고 점수")

print(f"\nnDCG@5: {retrieval_eval.ndcg:.3f}")
print(f"  -> 정답 2개(1,3번째)가 상위에 있어 좋은 점수")

print(f"\nKeyword Coverage: {retrieval_eval.keyword_coverage:.1%}")
print(f"  -> 4개 키워드 중 {int(retrieval_eval.keyword_coverage * 4)}개 발견")

print(f"\n관련 문서 발견: {retrieval_eval.relevant_found}/{len(test_case['relevant_docs'])}개")
print(f"\n종합 점수: {retrieval_eval.overall_score:.3f}")

In [ ]:
# 답변 평가 실행 (LLM API 호출)
# 주의: 이 셀은 API 키가 설정되어 있어야 실행됩니다.

try:
    answer_eval = evaluate_answer_with_llm(
        question=test_case["question"],
        generated_answer=rag_output["generated_answer"],
        reference_answer=test_case["reference_answer"],
        model="gpt-4o-mini"
    )
    
    print("=" * 60)
    print("답변 평가 결과 (Answer Evaluation by LLM Judge)")
    print("=" * 60)
    print(f"\n정확도 (Accuracy): {answer_eval.accuracy}/5")
    print(f"완전성 (Completeness): {answer_eval.completeness}/5")
    print(f"관련성 (Relevance): {answer_eval.relevance}/5")
    print(f"\n평균 점수: {answer_eval.average_score:.2f}/5")
    print(f"정규화 점수: {answer_eval.normalized_score:.2f}")
    print(f"\n피드백: {answer_eval.feedback}")
    
except Exception as e:
    print(f"API 호출 실패: {e}")
    print("\nAPI 키를 확인하거나, 아래 모의 결과를 참고하세요:")
    
    # 모의 결과
    mock_eval = AnswerEval(
        accuracy=4,
        completeness=3,
        relevance=5,
        feedback="답변이 정확하고 관련성이 높지만, sorted() 함수에 대한 설명이 누락되어 완전성이 다소 낮습니다."
    )
    print(f"\n[모의 결과]")
    print(f"정확도: {mock_eval.accuracy}/5")
    print(f"완전성: {mock_eval.completeness}/5")
    print(f"관련성: {mock_eval.relevance}/5")
    print(f"피드백: {mock_eval.feedback}")

---

## 6. 종합 평가 리포트 클래스

In [ ]:
class RAGEvaluationReport(BaseModel):
    """
    RAG 시스템 종합 평가 리포트
    """
    question_id: str
    question: str
    category: QuestionCategory
    retrieval: RetrievalEval
    answer: Optional[AnswerEval] = None
    
    def print_report(self):
        """평가 리포트 출력"""
        print("\n" + "=" * 70)
        print(f"RAG 평가 리포트: {self.question_id}")
        print("=" * 70)
        
        print(f"\n질문: {self.question}")
        print(f"카테고리: {self.category}")
        
        print("\n[검색 평가]")
        print(f"  MRR: {self.retrieval.mrr:.3f}")
        print(f"  nDCG: {self.retrieval.ndcg:.3f}")
        print(f"  Keyword Coverage: {self.retrieval.keyword_coverage:.1%}")
        print(f"  종합: {self.retrieval.overall_score:.3f}")
        
        if self.answer:
            print("\n[답변 평가]")
            print(f"  정확도: {self.answer.accuracy}/5")
            print(f"  완전성: {self.answer.completeness}/5")
            print(f"  관련성: {self.answer.relevance}/5")
            print(f"  평균: {self.answer.average_score:.2f}/5")
            print(f"\n  피드백: {self.answer.feedback}")
        
        print("\n" + "=" * 70)


# 종합 리포트 생성 예시
report = RAGEvaluationReport(
    question_id="q001",
    question=test_case["question"],
    category="direct_fact",
    retrieval=retrieval_eval,
    answer=AnswerEval(
        accuracy=4,
        completeness=3,
        relevance=5,
        feedback="답변이 정확하고 관련성이 높지만, sorted() 함수 설명이 누락됨"
    )
)

report.print_report()

---

## 7. 결론 및 Best Practices

### 핵심 정리

| 평가 단계 | 주요 지표 | 목적 |
|----------|----------|------|
| **검색 평가** | MRR, nDCG, Coverage | 올바른 문서를 찾았는가? |
| **답변 평가** | Accuracy, Completeness, Relevance | 올바른 답변을 생성했는가? |

### Best Practices

1. **평가 자동화**
   - CI/CD 파이프라인에 평가 포함
   - 배포 전 성능 회귀 테스트

2. **다양한 질문 카테고리**
   - direct_fact만으로는 부족
   - comparative, spanning 등 어려운 질문도 테스트

3. **정기적인 평가**
   - 문서 업데이트 시 재평가
   - 모델 변경 시 비교 평가

4. **평가 데이터 품질**
   - 전문가 검토된 테스트 셋
   - 실제 사용자 질문 반영

5. **개선 루프 구축**
   - 평가 -> 분석 -> 개선 -> 재평가
   - 실패 케이스 상세 분석

### 다음 단계

1. 더 큰 테스트 셋 구축 (100개 이상)
2. Ragas, DeepEval 등 전문 평가 프레임워크 활용
3. Human evaluation과 LLM evaluation 비교
4. 평가 대시보드 구축

---

## 참고 자료

- [Ragas: RAG 평가 프레임워크](https://github.com/explodinggradients/ragas)
- [DeepEval: LLM 평가 도구](https://github.com/confident-ai/deepeval)
- [LangChain Evaluation](https://python.langchain.com/docs/guides/evaluation)
- [BEIR Benchmark](https://github.com/beir-cellar/beir)

---

**이 노트북을 완료하셨습니다!**

이제 여러분은:
- 테스트 데이터를 설계할 수 있습니다
- 검색 품질을 MRR, nDCG로 평가할 수 있습니다
- LLM-as-a-Judge로 답변 품질을 평가할 수 있습니다
- 종합 평가 리포트를 생성할 수 있습니다